In [ ]:
!gdown --id 1-AlW7oNJHaqi3xk_9dWHUS52Dzl_FmFW

In [ ]:
!gdown --id 1-8TsrqTRFP-q9TM-6HinhO0ZVXFHq9TB

In [64]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [65]:
import pandas as pd
import numpy as np

In [66]:
import os

# Define the folder path
folder_path = '/content/drive/MyDrive/bootcamp/data'

# Create the folder if it doesn't already exist
os.makedirs(folder_path, exist_ok=True)

In [ ]:
import shutil

# Move train_data.csv
shutil.move('/content/train_data.csv', f'{folder_path}/train_data.csv')

# Move test_data.csv
shutil.move('/content/test_data.csv', f'{folder_path}/test_data.csv')

In [67]:
train_data_path = f'{folder_path}/train_data.csv'
test_data_path = f'{folder_path}/test_data.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

<ipython-input-67-e081c316124a>:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(train_data_path)


In [68]:
train_data.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,2,NaN,False,2016-11-11,A2OSUEZJIN7BI,0511189877,NaN,Chris,I have an older URC-WR7 remote and thought thi...,Cannot Learn,1478822400
1,5,NaN,True,2016-06-06,A2NETQRG6JHIG7,0511189877,NaN,Qrysta White,First time I've EVER had a remote that needed ...,zero programming needed! Miracle!?,1465171200
2,4,NaN,True,2016-03-10,A12JHGROAX49G7,0511189877,NaN,Linwood,Got them and only 2 of them worked. company ca...,Works Good and programs easy.,1457568000
3,5,NaN,True,2016-01-14,A1KV65E2TMMG6F,0511189877,NaN,Dane Williams,I got tired of the remote being on the wrong s...,Same as TWC remote,1452729600
4,5,NaN,True,2016-10-20,A280POPEWI0NSA,0594459451,NaN,Kristina H.,After purchasing cheap cords from another webs...,Good Quality Cord,1476921600


In [69]:
train_data.shape

(838944, 11)

In [70]:
train_data['reviewText'][0]

"I have an older URC-WR7 remote and thought this would be an upgrade (and because TWC stuck me with it), but this one fails where the other one didn't.  The old remote could go head to head and learn.  I have 2 different Blu Ray players (LG & Panasonic) and this one fails on both.  It cannot learn buttons.  The biggest problem is in streaming when I need to hit the blue, red, yellow, green buttons.  These cannot work either of them, so I have to pull out the remote for that one setting.  I don't know why they give you multiple code methods but no learning."

In [71]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 838944 entries, 0 to 838943
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         838944 non-null  int64 
 1   vote            191468 non-null  object
 2   verified        838944 non-null  bool  
 3   reviewTime      838944 non-null  object
 4   reviewerID      838944 non-null  object
 5   asin            838944 non-null  object
 6   style           490613 non-null  object
 7   reviewerName    838717 non-null  object
 8   reviewText      838944 non-null  object
 9   summary         838868 non-null  object
 10  unixReviewTime  838944 non-null  int64 
dtypes: bool(1), int64(2), object(8)
memory usage: 64.8+ MB


In [72]:
train_data['reviewText'] = train_data['reviewText'].str.lower().str.strip()
train_data['summary'] = train_data['summary'].str.lower().str.strip()

test_data['reviewText'] = test_data['reviewText'].str.lower().str.strip()
test_data['summary'] = test_data['summary'].str.lower().str.strip()

In [73]:
import re

# Define a regex pattern that removes all punctuation except for ?! and .
pattern = r'[^a-zA-Z0-9\s?!\.]'

# Apply this to the reviewText and summary columns, with a check for string type
train_data['reviewText'] = train_data['reviewText'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)
train_data['summary'] = train_data['summary'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)

# Assuming you have a similar structure for test_data
test_data['reviewText'] = test_data['reviewText'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)
test_data['summary'] = test_data['summary'].apply(lambda x: re.sub(pattern, '', x) if isinstance(x, str) else x)

In [74]:
!pip install textblob

In [ ]:
from textblob import TextBlob

def correct_spelling(text):
    return str(TextBlob(text).correct())

train_data['reviewText'] = train_data['reviewText'].apply(correct_spelling)
train_data['summary'] = train_data['summary'].apply(correct_spelling)

test_data['reviewText'] = test_data['reviewText'].apply(correct_spelling)
test_data['summary'] = test_data['summary'].apply(correct_spelling)

In [75]:
!pip install symspellpy

In [76]:
import pkg_resources
from symspellpy import SymSpell, Verbosity

In [77]:
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

input_term = "memebers"

suggestions = sym_spell.lookup(input_term, Verbosity.CLOSEST, max_edit_distance=2)
for suggestion in suggestions:
    print(suggestion)

members, 1, 226656153


In [78]:
from operator import attrgetter

dictionary_cache = set()


def find_nearest_word_in_dict(word):
    word = word.lower()

    if word in dictionary_cache:
        return word

    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)
    suggestions = sorted(suggestions, key=attrgetter('distance'))

    min_distance = suggestions[0].distance
    nearest_word = max([s for s in suggestions if s.distance == min_distance], key=attrgetter('count'))

    dictionary_cache.add(nearest_word.term)

    return nearest_word.term

In [79]:
sample_sentence = train_data['reviewText'][9]
sample_sentence

'brand name adapter this is just like the one you got when you bought your nook. i would always recommend using a designated adapter. the charging cable is a whole different story but the adapter defiantly use the designated charger.'

In [80]:
tokens = sample_sentence.split()
tokens

['brand',
 'name',
 'adapter',
 'this',
 'is',
 'just',
 'like',
 'the',
 'one',
 'you',
 'got',
 'when',
 'you',
 'bought',
 'your',
 'nook.',
 'i',
 'would',
 'always',
 'recommend',
 'using',
 'a',
 'designated',
 'adapter.',
 'the',
 'charging',
 'cable',
 'is',
 'a',
 'whole',
 'different',
 'story',
 'but',
 'the',
 'adapter',
 'defiantly',
 'use',
 'the',
 'designated',
 'charger.']

In [81]:
corrected_tokens = []

for t in tokens:
    nearest = find_nearest_word_in_dict(t)
    corrected_tokens.append(nearest)

### Mis-spelling correction example

In [82]:
for t, c in zip(tokens, corrected_tokens):
    print(t, c)

brand brand
name name
adapter adapter
this this
is is
just just
like like
the the
one one
you you
got got
when when
you you
bought bought
your your
nook. nook
i i
would would
always always
recommend recommend
using using
a a
designated designated
adapter. adapter
the the
charging charging
cable cable
is is
a a
whole whole
different different
story story
but but
the the
adapter adapter
defiantly defiantly
use use
the the
designated designated
charger. charger


In [83]:
def correct_spelling_symspell(sentence):
    tokens = sentence.split()
    corrected_tokens = list(map(find_nearest_word_in_dict, tokens))

    return ' '.join(corrected_tokens)

In [84]:
corrected_sentence = correct_spelling_symspell(sample_sentence)

print('main sentence: \n{}'.format(sample_sentence))
print('corrected sentence: \n{}'.format(corrected_sentence))

main sentence: 
brand name adapter this is just like the one you got when you bought your nook. i would always recommend using a designated adapter. the charging cable is a whole different story but the adapter defiantly use the designated charger.
corrected sentence: 
brand name adapter this is just like the one you got when you bought your nook i would always recommend using a designated adapter the charging cable is a whole different story but the adapter defiantly use the designated charger


### Removing Stop words from review text and summaries

In [85]:
!pip install nltk

In [86]:
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag, word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [87]:
pos_tag(word_tokenize("John's very big idea isn't all that bad."))

[('John', 'NNP'),
 ("'s", 'POS'),
 ('very', 'RB'),
 ('big', 'JJ'),
 ('idea', 'NN'),
 ('is', 'VBZ'),
 ("n't", 'RB'),
 ('all', 'PDT'),
 ('that', 'DT'),
 ('bad', 'JJ'),
 ('.', '.')]

In [88]:
stop_words = set(stopwords.words('english'))


def remove_stop_words(text):
    tokens = word_tokenize(text.lower())
    word_tag = pos_tag(tokens)

    tokens = [w[0] for w in word_tag if w[0] not in stop_words or w[1] in ['VBZ', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'RB', 'RBR', 'RBS']]

    return ' '.join(tokens)

In [89]:
print('main sentence: \n{}\n'.format(sample_sentence))
print('sentence after removing stop words:\n{}'.format(remove_stop_words(sample_sentence)))

main sentence: 
brand name adapter this is just like the one you got when you bought your nook. i would always recommend using a designated adapter. the charging cable is a whole different story but the adapter defiantly use the designated charger.

sentence after removing stop words:
brand name adapter is just like one got bought nook . would always recommend using designated adapter . charging cable is whole different story adapter defiantly use designated charger .


In [90]:
train_data['overall'].value_counts()

,count
overall,
5,461485
4,156514
1,82950
3,81239
2,56756


In [ ]:
train_data_sample = train_data.sample(frac=0.2)
train_data_sample.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
691413,5,5,True,2017-05-05,A2WM4XD9GQW4D9,B01EK7TEL4,{'Style:': ' Sound Bar with Subwoofer'},Pascal Renault,fit in a tight spot no room for receiver and s...,impressive for the price,1493942400
128283,1,NaN,True,2017-02-03,AOKQ5KDS3LRRK,B007IL6OR2,{'Package Type:': ' Standard Packaging'},Alex,product is defective connection only works on ...,only half works got a defective item.,1486080000
306502,5,NaN,True,2016-02-03,AT4MNYYZWM907,B00LSZTCO6,{'Color:': ' Silver'},Adam Hume,amazing product. i bought a different bluetoo...,amazing product. i bought a different bluetoot...,1454457600
449923,4,4,True,2017-02-01,AN17Z00NUJW0V,B00XI87KV8,{'Style:': ' 50-Inch Tripod'},Kate Mahoney,for the price this is a great little tripod an...,great starterlight use tripod for audio video ...,1485907200
763986,5,NaN,True,2016-08-29,A2OFBWUFQOMEOP,B00AW54YWS,NaN,Anthony Miller,i have had this camcorder for about 2 years an...,the battery still holds a good charge and last...,1472428800


In [ ]:
train_data_sample['overall'].value_counts()

,count
overall,
5,92457
4,31354
1,16479
3,16203
2,11296


In [ ]:
train_data_sample['reviewText'] = train_data_sample['reviewText'].apply(remove_stop_words)

In [91]:
test_data.head()

,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,NaN,True,2018-06-04,A20GGWE66JW9X2,B006Z394GM,{'Color:': ' FPS01-C'},Brian C Toner,the name and description of this device are mi...,the prize for most useless invention of all ti...,1528070400
1,NaN,True,2018-02-18,ARYJWXHEYHG9M,B005BE058W,"{'Size:': ' 1000W', 'Style:': ' G2'}",Snake,one of the molex connectors on the power suppl...,melted molex connector,1518912000
2,NaN,True,2018-01-20,A10LHZ7WFZ7HLL,B01DA0YCNC,NaN,Amazon Customer,remote constantly disconnects roku player free...,bricked on the regular,1516406400
3,NaN,True,2018-06-25,A11VN8EOHNLP72,B00FBJ4KYC,NaN,Jeremy Bray,i purchased this 4 year protection plan for a ...,do not buy!!!,1529884800
4,3.0,True,2016-08-17,A194Y8P8TVT7P9,B00P7G82TS,NaN,Mark,i bought one of these and have regretted it ev...,nightmare dont buy,1471392000


In [ ]:
test_data['reviewText'] = test_data['reviewText'].apply(remove_stop_words)

In [ ]:
train_data_sample.to_csv('train_data_sample.csv')
shutil.move('/content/train_data_sample.csv', f'{folder_path}/train_data_sample.csv')

'/content/drive/MyDrive/bootcamp/data/train_data_sample.csv'

### Spell checking after removing stop words in Train data sample

In [92]:
train_data_sample_path = f'{folder_path}/train_data_sample.csv'
train_data_sample = pd.read_csv(train_data_sample_path)

<ipython-input-92-e5d03b372bc2>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data_sample = pd.read_csv(train_data_sample_path)


In [93]:
train_data_sample.head()

,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,44867,4,NaN,True,2016-12-27,A2SGDGD3R8YAR3,B0015HOFZI,NaN,Sean&#039;s Amazon site,good price money . really like turntable works...,good turntable for a reasonble price. softwar...,1482796800
1,514712,3,NaN,True,2016-09-15,A31XQAQ6OAK63H,B013QJ0W8Q,{'Color:': ' Advanced Car Kits'},Heather,didnt realize wouldnt be able have radio have ...,okay for music not so much for phone calls,1473897600
2,321051,1,NaN,True,2016-11-11,A1VNJUKIW0DROT,B00MSTG93S,NaN,Luis Alberto Carranza Gonzalez,bought march 24 i travel europe october 10 dev...,it failed 7 months after i bought it.,1478822400
3,643355,5,2,False,2016-11-12,A194OCP49BPKN,B01BY7YOAY,NaN,Jimbo,hi first got denon avrs510bt receiver hooked p...,yamaha rxv381bl vs denon avrs510bt,1478908800
4,470838,5,NaN,True,2016-05-03,A41130X01BBBT,B00ZI7IS2W,{'Color:': ' 8CH+8Camera'},Gabby1947,good system works great . installed disabled o...,great product great price,1462233600


In [94]:
train_data_sample.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
train_data_sample['reviewText'] = train_data_sample['reviewText'].apply(correct_spelling_symspell)

In [ ]:
test_data['reviewText'] = test_data['reviewText'].apply(correct_spelling_symspell)

In [ ]:
import shutil

train_data_sample.to_csv('train_data_sample_spell_checked.csv')
shutil.move('/content/train_data_sample_spell_checked.csv', f'{folder_path}/train_data_sample_spell_checked.csv')

'/content/drive/MyDrive/bootcamp/data/train_data_sample_spell_checked.csv'

In [95]:
train_data_sample_spell_checked_path = f'{folder_path}/train_data_sample_spell_checked.csv'
train_data_sample_spell_checked = pd.read_csv(train_data_sample_spell_checked_path)

train_data_sample_spell_checked.head()

<ipython-input-95-f699217def39>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data_sample_spell_checked = pd.read_csv(train_data_sample_spell_checked_path)


,Unnamed: 0,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,0,4,NaN,True,2016-12-27,A2SGDGD3R8YAR3,B0015HOFZI,NaN,Sean&#039;s Amazon site,good price money a really like turntable works...,good turntable for a reasonble price. softwar...,1482796800
1,1,3,NaN,True,2016-09-15,A31XQAQ6OAK63H,B013QJ0W8Q,{'Color:': ' Advanced Car Kits'},Heather,didst realize wouldn't be able have radio have...,okay for music not so much for phone calls,1473897600
2,2,1,NaN,True,2016-11-11,A1VNJUKIW0DROT,B00MSTG93S,NaN,Luis Alberto Carranza Gonzalez,bought march of i travel europe october of dev...,it failed 7 months after i bought it.,1478822400
3,3,5,2,False,2016-11-12,A194OCP49BPKN,B01BY7YOAY,NaN,Jimbo,hi first got devon avrs510bt receiver hooked p...,yamaha rxv381bl vs denon avrs510bt,1478908800
4,4,5,NaN,True,2016-05-03,A41130X01BBBT,B00ZI7IS2W,{'Color:': ' 8CH+8Camera'},Gabby1947,good system works great a installed disabled o...,great product great price,1462233600


In [96]:
train_data_sample_spell_checked.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
import shutil

test_data.to_csv('test_data_spell_checked.csv')
shutil.move('/content/test_data_spell_checked.csv', f'{folder_path}/test_data_spell_checked.csv')

'/content/drive/MyDrive/bootcamp/data/test_data_spell_checked.csv'

In [97]:
test_data_spell_checked_path = f'{folder_path}/test_data_spell_checked.csv'
test_data_spell_checked = pd.read_csv(test_data_spell_checked_path)

test_data_spell_checked.head()

,Unnamed: 0,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
0,0,NaN,True,2018-06-04,A20GGWE66JW9X2,B006Z394GM,{'Color:': ' FPS01-C'},Brian C Toner,name description device are misleading a is su...,the prize for most useless invention of all ti...,1528070400
1,1,NaN,True,2018-02-18,ARYJWXHEYHG9M,B005BE058W,"{'Size:': ' 1000W', 'Style:': ' G2'}",Snake,one rolex connectors power supply started melt...,melted molex connector,1518912000
2,2,NaN,True,2018-01-20,A10LHZ7WFZ7HLL,B01DA0YCNC,NaN,Amazon Customer,remote constantly disconnects roku player free...,bricked on the regular,1516406400
3,3,NaN,True,2018-06-25,A11VN8EOHNLP72,B00FBJ4KYC,NaN,Jeremy Bray,purchased a year protection plan security syst...,do not buy!!!,1529884800
4,4,3.0,True,2016-08-17,A194Y8P8TVT7P9,B00P7G82TS,NaN,Mark,bought one have regretted ever since a gas rec...,nightmare dont buy,1471392000


In [98]:
test_data_spell_checked.drop('Unnamed: 0', axis=1, inplace=True)

### Finding Similary words in two separate dataframes

In [99]:
!pip install gensim

In [100]:
from sklearn.metrics.pairwise import cosine_similarity
import gensim.downloader as api
from tqdm import tqdm

# Load a simpler pre-trained word embedding model (GloVe or Word2Vec)
word_vectors = api.load("glove-wiki-gigaword-50")  # Can replace with "word2vec-google-news-300"

# Define target word embeddings for "guarantee" and "warranty"
target_words = ["guarantee", "warranty"]
target_embeddings = {word: word_vectors[word] for word in target_words}

# Quick keyword-based filter
keywords = set(target_words)

# Function to classify based on keyword presence and cosine similarity
def classify_review_simple(review, threshold=0.8):
    # Step 1: Quick filter - Check if any keyword exists in the review
    if any(keyword in review.lower() for keyword in keywords):
        words = review.lower().split()
        for word in words:
            if word in word_vectors:
                # Calculate similarity to target words
                word_embedding = word_vectors[word].reshape(1, -1)
                for target_word, target_embedding in target_embeddings.items():
                    similarity = cosine_similarity(word_embedding, target_embedding.reshape(1, -1))[0][0]
                    if similarity >= threshold:
                        return "guarantee"
    return "no guarantee"

# Apply classification with progress tracking
tqdm.pandas(desc="Classifying reviews")  # Set up tqdm with description

# Classify all reviews with progress bar
train_data_sample_spell_checked['classification'] = train_data_sample_spell_checked['reviewText'].progress_apply(classify_review_simple)

# Split into DataFrames based on classification
guarantee_reviews_train = train_data_sample_spell_checked[train_data_sample_spell_checked['classification'] == "guarantee"]
no_guarantee_reviews_train = train_data_sample_spell_checked[train_data_sample_spell_checked['classification'] == "no guarantee"]

Classifying reviews: 100%|██████████| 167789/167789 [06:01<00:00, 463.81it/s]


In [ ]:
import ace_tools as tools

tools.display_dataframe_to_user(name="Guarantee Reviews in Train Data", dataframe=guarantee_reviews_train)
tools.display_dataframe_to_user(name="No Guarantee Reviews in Train Data", dataframe=no_guarantee_reviews_train)

In [101]:
train_data_sample_spell_checked.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,classification
0,4,NaN,True,2016-12-27,A2SGDGD3R8YAR3,B0015HOFZI,NaN,Sean&#039;s Amazon site,good price money a really like turntable works...,good turntable for a reasonble price. softwar...,1482796800,no guarantee
1,3,NaN,True,2016-09-15,A31XQAQ6OAK63H,B013QJ0W8Q,{'Color:': ' Advanced Car Kits'},Heather,didst realize wouldn't be able have radio have...,okay for music not so much for phone calls,1473897600,no guarantee
2,1,NaN,True,2016-11-11,A1VNJUKIW0DROT,B00MSTG93S,NaN,Luis Alberto Carranza Gonzalez,bought march of i travel europe october of dev...,it failed 7 months after i bought it.,1478822400,guarantee
3,5,2,False,2016-11-12,A194OCP49BPKN,B01BY7YOAY,NaN,Jimbo,hi first got devon avrs510bt receiver hooked p...,yamaha rxv381bl vs denon avrs510bt,1478908800,no guarantee
4,5,NaN,True,2016-05-03,A41130X01BBBT,B00ZI7IS2W,{'Color:': ' 8CH+8Camera'},Gabby1947,good system works great a installed disabled o...,great product great price,1462233600,no guarantee


### Loading Brand and title data

In [102]:
!gdown --id 1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1I9aPAvvYgQWdHGKtnd7IeTGXpx8vOm4h
To: /content/title_brand.csv
100% 97.3M/97.3M [00:03<00:00, 30.5MB/s]


In [ ]:
shutil.move('/content/title_brand.csv', f'{folder_path}/title_brand.csv')

'/content/drive/MyDrive/bootcamp/data/title_brand.csv'

In [104]:
title_brand_path = f'{folder_path}/title_brand.csv'
title_brand = pd.read_csv(title_brand_path)

title_brand.head()

,asin,title,brand
0,0011300000,Genuine Geovision 1 Channel 3rd Party NVR IP S...,GeoVision
1,0043396828,"Books ""Handbook of Astronomical Image Processi...",33 Books Co.
2,0060009810,One Hot Summer,Visit Amazon's Carolina Garcia Aguilera Page
3,0060219602,Hurray for Hattie Rabbit: Story and pictures (...,Visit Amazon's Dick Gackenbach Page
4,0060786817,sex.lies.murder.fame.: A Novel,Visit Amazon's Lolita Files Page


In [105]:
title_brand[title_brand['asin'] == 'B0015HOFZI']

,asin,title,brand
125368,B0015HOFZI,Sony PSLX300USB USB Stereo Turntable,Sony


In [106]:
train_data_sample_spell_checked = train_data_sample_spell_checked.merge(title_brand, on='asin', how='inner')
train_data_sample_spell_checked.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,classification,title,brand
0,4,NaN,True,2016-12-27,A2SGDGD3R8YAR3,B0015HOFZI,NaN,Sean&#039;s Amazon site,good price money a really like turntable works...,good turntable for a reasonble price. softwar...,1482796800,no guarantee,Sony PSLX300USB USB Stereo Turntable,Sony
1,3,NaN,True,2016-09-15,A31XQAQ6OAK63H,B013QJ0W8Q,{'Color:': ' Advanced Car Kits'},Heather,didst realize wouldn't be able have radio have...,okay for music not so much for phone calls,1473897600,no guarantee,"TaoTronics Bluetooth Receiver/Car Kit, Portabl...",TaoTronics
2,1,NaN,True,2016-11-11,A1VNJUKIW0DROT,B00MSTG93S,NaN,Luis Alberto Carranza Gonzalez,bought march of i travel europe october of dev...,it failed 7 months after i bought it.,1478822400,guarantee,BESTEK Power Step Down 220V to 110V Voltage Co...,BESTEK
3,5,2,False,2016-11-12,A194OCP49BPKN,B01BY7YOAY,NaN,Jimbo,hi first got devon avrs510bt receiver hooked p...,yamaha rxv381bl vs denon avrs510bt,1478908800,no guarantee,Yamaha RX-V381BL Receiver (Black),Yamaha Audio
4,5,NaN,True,2016-05-03,A41130X01BBBT,B00ZI7IS2W,{'Color:': ' 8CH+8Camera'},Gabby1947,good system works great a installed disabled o...,great product great price,1462233600,no guarantee,Funlux Waterproof 8 Channel 960H HDMI DVR Hom...,Funlux


In [107]:
def embed_product_info_in_comment(record):
    return 'my idea about product with id \'{}\' with name {} from brand {} is: {}'.format(record['asin'], record['title'], record['brand'], record['reviewText'])

In [108]:
result = embed_product_info_in_comment(train_data_sample_spell_checked.iloc[0])
result

"my idea about product with id 'B0015HOFZI' with name Sony PSLX300USB USB Stereo Turntable from brand Sony is: good price money a really like turntable works well connected ria jacks sounder a comes sony sound forge software is little bit complicated a was able figure record vinyl records laptop burn cd a was fairly easy process a recording onto cd was plenty loud did find recording doesn't always start beginning track a am still experimenting report further software future a software prompts settings make sure set line get best recording a has been years since used turntable did not immediately see belt is wrapped around bottom turntable simply pulling belt capstan allows hook turntable a visual directions were not good a"

In [109]:
train_data_sample_spell_checked['reviewText'] = train_data_sample_spell_checked.apply(embed_product_info_in_comment, axis=1)

In [110]:
train_data_sample_spell_checked['reviewText'].head()

,reviewText
0,my idea about product with id 'B0015HOFZI' wit...
1,my idea about product with id 'B013QJ0W8Q' wit...
2,my idea about product with id 'B00MSTG93S' wit...
3,my idea about product with id 'B01BY7YOAY' wit...
4,my idea about product with id 'B00ZI7IS2W' wit...


In [118]:
import shutil

train_data_sample_spell_checked.to_csv('train_data_sample_spell_checked_clean.csv')
shutil.move('/content/train_data_sample_spell_checked_clean.csv', f'{folder_path}/train_data_sample_spell_checked_clean.csv')

'/content/drive/MyDrive/bootcamp/data/train_data_sample_spell_checked_clean.csv'

### Loading cleaned data and Model and training on user comments dataset

In [120]:
train_data_clean_path = f'{folder_path}/train_data_sample_spell_checked_clean.csv'
train_data_clean = pd.read_csv(train_data_clean_path)
train_data_clean.drop('Unnamed: 0', axis=1, inplace=True)

<ipython-input-120-0ebcfce2c606>:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data_clean = pd.read_csv(train_data_clean_path)
